# ResponseGetData

> preferred response class for all API requests

In [17]:
#| default_exp ResponseGetData

In [18]:
#| hide
from nbdev.showdoc import show_doc

In [19]:
#| export
from dataclasses import dataclass, field
from fastcore.utils import patch_to
from fastcore.test import test_eq

import requests
import asyncio
import aiohttp
from typing import  Union


All API routes should return an instance of `ResponseGetData`.

In [20]:
#| export

@dataclass
class ResponseGetData:
    """preferred response class for all API routes"""
    status: int
    response: Union[list, dict, str]
    is_success: bool
    auth: dict = field(default_factory=dict)

#### Sample Implementation of ResponseGetData

In [21]:
rgd = ResponseGetData(status = 200, response = "test", is_success = True)
rgd

ResponseGetData(status=200, response='test', is_success=True, auth={})

In [22]:
#| hide
test_eq(rgd.is_success , True)

# Generate ResponseGetData with classmethods

Users can generate an instance of the `ResponseGetData` class from two private classmethods (see below).

This class currently supports implementations of two API request libraries, `requests.request` (synchronous) and `aiohttp.ClientRequest` (asynchronous).

In [23]:
#| export
@patch_to(ResponseGetData, cls_method = True)
def _from_requests_response(cls, res : requests.Response #requests response object
                           ) -> ResponseGetData:
    """returns ResponseGetData"""
    
    #JSON responses
    if res.ok and "application/json" in res.headers.get("Content-Type", {}):
        return ResponseGetData(
            status = res.status_code, 
            response = res.json(),
            is_success=True
        )

    #default text responses
    elif res.ok:
        return ResponseGetData(
            status=res.status_code,
            response=res.text,
            is_success=True
        )
    
    # errors
    return ResponseGetData(
        status=res.status_code,
        response=res.reason,
        is_success=False
    )

In [24]:
show_doc(ResponseGetData._from_requests_response)

---

[source](https://github.com/jaewilson07/nbdev_domo/blob/main/nbdev_domo/ResponseGetData.py#L28){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_requests_response

>      ResponseGetData._from_requests_response (res:requests.models.Response)

returns ResponseGetData

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| res | Response | requests response object |
| **Returns** | **ResponseGetData** |  |

#### sample implementation of _from_requests_response

In [35]:
test_url = 'http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita'

res = requests.request(method = 'GET', url = test_url)

test_res = ResponseGetData._from_requests_response(res)
test_res.__dict__.keys()

dict_keys(['status', 'response', 'is_success', 'auth'])

In [26]:
#| hide
test_eq(isinstance( test_res, ResponseGetData), True)

In [27]:
#| export
@patch_to(ResponseGetData, cls_method = True)
async def _from_aiohttp_response(cls, res : aiohttp.ClientResponse #requests response object
                           ) -> ResponseGetData:
    
    """async method returns ResponseGetData"""
    
    if res.ok and "application/json" in res.headers.get("Content-Type", {}) :
        try:
            return ResponseGetData(
                status=res.status, response=await res.json(), is_success=True
            )

        # handle if unable to decode json()
        except asyncio.TimeoutError as e:
            print(e)
            print("response included json, but defaulted to backup decode method")

            return ResponseGetData(
                status=res.status, response=await res.json(content_type=None), is_success=True
            )

        # response is text
    elif res.ok:
        return ResponseGetData(
            status=res.status, response=await res.text(), is_success=True
        )

    # response is error
    else:
        return ResponseGetData(status=res.status, response=str(res.reason), is_success=False)

In [28]:
show_doc(ResponseGetData._from_aiohttp_response)

---

[source](https://github.com/jaewilson07/nbdev_domo/blob/main/nbdev_domo/ResponseGetData.py#L57){target="_blank" style="float:right; font-size:smaller"}

### ResponseGetData._from_aiohttp_response

>      ResponseGetData._from_aiohttp_response
>                                              (res:aiohttp.client_reqrep.Client
>                                              Response)

async method returns ResponseGetData

#### sample implementation of _from_aiohttp_response.
Notice the use of async / await

In [39]:
test_url = 'http://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita'

timeout = aiohttp.ClientTimeout(total = 10)
session = aiohttp.ClientSession(timeout=timeout)

try:
    res = await session.get(url = test_url)
    
    test_res = await ResponseGetData._from_aiohttp_response(res)
    print(test_res.__dict__.keys())
except TimeoutError as e:
    # sometimes aiohttp errors out for unexpected reasons.
    print(e)

finally:
    await session.close()


dict_keys(['status', 'response', 'is_success', 'auth'])


In [30]:
#| hide
test_eq(isinstance( test_res, ResponseGetData), True)

In [31]:
#| hide
import nbdev; nbdev.nbdev_export()